In [1]:
import pandas as pd
import json
import os
import numpy as np
import re

## 读取数据

In [2]:
io = 'post_1689646329899.xlsx'

In [3]:
data = pd.read_excel(io)
data.head()

D:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,序号,姓名,档案号,初诊日期,性别,年龄,诊察医生,所属医院,待诊时间,舌像上传时间,AI结果返回时间,AI诊断修改时间,AI诊断结果,医生诊断结果,诊断状态,基础疾病,是否删除
0,1,张三,10002458,2023-07-14 16:45:39,男,25.0,揭凤鸣,测试医院1,2023-07-14 17:05:44,2023-07-14 17:37:10,2023-07-14 17:37:14,NaN,舌质\n舌尖红:有\n舌质老嫩:正常\n舌质齿痕:有\n舌质胖瘦:正常\n胖度值:7.5\n...,NaN,就诊,CKD3,正常
1,2,张三,10002458,2023-07-14 16:45:39,男,25.0,揭凤鸣,测试医院1,2023-07-14 16:45:39,2023-07-14 16:58:57,2023-07-14 16:59:02,2023-07-14 17:02:16,舌质\n舌尖红:有\n舌质老嫩:老\n舌质齿痕:有\n舌质胖瘦:胖大\n胖度值:8.3\n舌...,舌质\n舌尖红:有\n舌质老嫩:老\n舌质齿痕:有\n舌质胖瘦:胖大\n胖度值:8.3\n舌...,就诊,CKD3,正常
2,3,刘佳,10002235,2023-05-19 09:00:46,男,38.0,张晶晶,西苑中医院,2023-07-14 11:17:39,2023-07-14 11:18:01,2023-07-14 11:18:06,NaN,舌质\n舌尖红:有\n舌质老嫩:正常\n舌质齿痕:有\n舌质胖瘦:正常\n胖度值:7.4\n...,NaN,就诊,自汗,正常
3,4,刘一坤,10002457,2023-07-14 10:37:28,男,41.0,张晶晶,西苑中医院,2023-07-14 10:37:28,2023-07-14 10:37:47,2023-07-14 10:37:51,NaN,舌质\n舌尖红:无\n舌质老嫩:正常\n舌质齿痕:有\n舌质胖瘦:正常\n胖度值:6.6\n...,NaN,就诊,腰酸,正常
4,5,赵继香,10002456,2023-07-14 10:25:57,女,70.0,张晶晶,西苑中医院,2023-07-14 10:25:57,2023-07-14 10:26:17,2023-07-14 10:26:22,NaN,舌质\n舌尖红:无\n舌质老嫩:正常\n舌质齿痕:无\n舌质胖瘦:正常\n胖度值:6.5\n...,NaN,就诊,泌尿系感染,正常


In [4]:
data['AI诊断结果'][2]

'舌质\n舌尖红:有\n舌质老嫩:正常\n舌质齿痕:有\n舌质胖瘦:正常\n胖度值:7.4\n舌质颜色:淡红\n红度平均值:3.4\n瘀斑面积:0.0\n瘀点面积:0.1\n瘀点个数:3\n舌质裂纹:有\n\n舌苔\n舌苔颜色:淡黄\n黄度平均值:2.5\n剥落苔:无\n舌苔面积:78.02\n舌苔厚薄:厚\n厚度值:5.2\n舌苔腐腻:腻\n腐腻值:5.4\n舌苔润燥:润\n润度值:5.4\n糙苔:无\n糙度值:1.1\n\n舌下脉络\n脉络颜色:紫黑\n紫度平均值:9.8\n脉络粗细:0.12\n脉络长度:0.77\n脉络形态:正常'

## 查找AI与医生判断不同的样本

In [5]:
def getre(str, category):
    try:
        return re.search(category+r':.{1,2}', str).group()
    except :
        return 1
    
def get_diff(data, category, if_print=True):
    """
    获取某一个类别医生与AI判断不一致的样本信息
    category: 需要判断的类别
    """
    data['AI'] = data['AI诊断结果'].apply(lambda x:getre(x, category))
    data['医生'] = data['医生诊断结果'].apply(lambda x:getre(x, category))
    
    num = 0
    if if_print:
        print('档案号    医生           时间        医生诊断结果     AI诊断结果')
    for i in range(len(data['医生'])):
        if data['医生'][i] != data['AI'][i] and data['医生'][i] !=1:
            if if_print:
                print(data['档案号'][i], data['诊察医生'][i], data['舌像上传时间'][i], data['医生'][i], data['AI'][i])
            num = num + 1
    
    if if_print:
        print(num)
    return num

In [6]:
for i in ['老嫩', '舌苔颜色', '舌质颜色', '脉络颜色', '润燥',  '剥落苔', '舌质裂纹', '舌尖红', '腐腻', '舌质齿痕', '舌质裂纹',
          '脉络形态', '舌苔腐腻', '舌质胖瘦']:
    a = get_diff(data, i, if_print=False)
    print(i, a)

老嫩 112
舌苔颜色 125
舌质颜色 330
脉络颜色 372
润燥 90
剥落苔 63
舌质裂纹 39
舌尖红 56
腐腻 103
舌质齿痕 88
舌质裂纹 39
脉络形态 335
舌苔腐腻 103
舌质胖瘦 102


In [7]:
get_diff(data, '舌苔颜色')

档案号    医生           时间        医生诊断结果     AI诊断结果
10002359 徐国峰 2023-06-15 11:53:19 舌苔颜色:淡黄 舌苔颜色:白
10002364 徐国峰 2023-06-15 11:47:44 舌苔颜色:淡黄 舌苔颜色:白
10001177 徐国峰 2023-06-15 10:14:54 舌苔颜色:白 舌苔颜色:淡黄
10001759 徐国峰 2023-06-14 12:26:10 舌苔颜色:黄 舌苔颜色:白
10002405 徐国峰 2023-06-14 12:20:19 舌苔颜色:淡黄 舌苔颜色:白
10001857 徐国峰 2023-06-14 11:42:16 舌苔颜色:白 舌苔颜色:淡黄
10000439 徐国峰 2023-06-12 12:13:13 舌苔颜色:黄 舌苔颜色:白
10002069 徐国峰 2023-06-12 10:40:48 舌苔颜色:白 舌苔颜色:淡黄
10000328 徐国峰 2023-06-08 11:16:40 舌苔颜色:白 舌苔颜色:淡黄
10001177 徐国峰 2023-06-08 09:52:00 舌苔颜色:白 舌苔颜色:淡黄
10001678 徐国峰 2023-06-07 10:12:02 舌苔颜色:淡黄 舌苔颜色:白
10002186 徐国峰 2023-06-07 08:54:39 舌苔颜色:淡黄 舌苔颜色:黄
10001739 徐国峰 2023-06-05 16:33:42 舌苔颜色:白 舌苔颜色:淡黄
10002340 徐国峰 2023-06-05 12:47:30 舌苔颜色:淡黄 舌苔颜色:白
10002339 徐国峰 2023-06-05 12:35:46 舌苔颜色:白 舌苔颜色:灰黑
10001854 徐国峰 2023-06-05 11:46:12 舌苔颜色:黄 舌苔颜色:白
10002334 徐国峰 2023-06-05 10:56:46 舌苔颜色:白 舌苔颜色:淡黄
10002329 徐国峰 2023-06-05 09:30:34 舌苔颜色:淡黄 舌苔颜色:白
10002264 徐国峰 2023-05-31 11:16:40 舌苔颜色:黄 舌苔颜色:淡黄
10002250 徐国峰 2023-05-31 10:18:20 舌苔颜色:淡黄 舌苔

125


## 查找某一类

In [7]:
num = 0
for i in range(len(data['医生'])):
    try:
        if '灰黑' in data['医生诊断结果'][i]:
            print(data['档案号'][i], data['诊察医生'][i], data['舌像上传时间'][i], data['医生'][i], data['AI'][i])
            num = num + 1
    except:
        pass
print(num)

10002010 苏巧珍 2023-04-13 16:25:50 舌质胖瘦:正常 舌质胖瘦:正常
10000573 徐国峰 2023-04-12 08:46:14 舌质胖瘦:正常 舌质胖瘦:正常
10001742 徐国峰 2023-03-17 12:33:35 舌质胖瘦:正常 舌质胖瘦:正常
10001131 徐国峰 2023-02-17 11:22:55 舌质胖瘦:胖大 舌质胖瘦:胖大
10001068 老六 2023-01-07 17:14:28 舌质胖瘦:胖大 舌质胖瘦:正常
10000267 陆靖桥 2022-12-26 09:55:13 舌质胖瘦:瘦 舌质胖瘦:正常
6


In [28]:
len(data[data['AI诊断结果'].notnull() & data['医生诊断结果'].isnull()])

2203

In [20]:
from torchvision.models import vgg16

In [24]:
a = vgg16()

In [32]:
list(a.children())[0][0:8]

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [29]:
len(data[data['医生诊断结果'].notna()])

1129

In [30]:
len(data)

3765